In [15]:
import os
import urllib.request
import json
import pymysql.cursors
import pandas as pd
from pymongo import MongoClient
from itertools import chain
from riotwatcher import RiotWatcher
from tqdm import tqdm
from requests.exceptions import HTTPError
from converters.data2files import get_runes_reforged_json
from converters.data2frames import game_to_dataframe as g2df
from sqlalchemy import create_engine
from datetime import datetime as dt, timedelta
import pickle
from config.constants import LEAGUES_DATA_DICT, SQL_LEAGUES_CONN, MONGODB_CREDENTIALS, \
    API_KEY, SOLOQ, REGIONS, CUSTOM_PARTICIPANT_COLS, STANDARD_POSITIONS, SCRIMS_POSITIONS_COLS, \
    TOURNAMENT_GAME_ENDPOINT, SQL_LEAGUES_ENGINE, EXCEL_EXPORT_PATH_MERGED, EXPORTS_DIR, EXCEL_EXPORT_PATH, \
    RIFT_GAMES_QUEUES, SLO, TOURNAMENT_TL_ENDPOINT, SQL_EXPORTS_ENGINE